# Webscrape

### Simple (initial) Version

In [2]:
# Simple Version
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get('https://www.erowid.org/experiences/exp.cgi?S1=2')

data_list=[]

view_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'exp.php?ID=')]")
for idx, view in enumerate(view_links):
    href = view.get_attribute('href')
    view_links[idx] = href

# For each link get the trip text and add the text and link to a list of dicts
for href in view_links:
    driver.get(href)
    trip_text = driver.find_elements(By.CLASS_NAME, 'report-text-surround')
    for trip in trip_text:
        text = trip.text.encode('utf-8').decode('utf-8')
        print(text)
        data_list.append({'Text': text, 'Link': href})

# Update the dataframe with the collected data
df = pd.DataFrame(data_list)
df["Drug"] = 'LSD'
df.to_csv("LSD_Trip_Reports.csv")

  DOSE:
T+ 0:00 200 ug sublingual LSD (blotter / tab)
  T+ 1:30 1 bowl smoked Cannabis (flowers)
  T+ 2:00 1 glass oral Coffee  
  T+ 0:00   smoked Tobacco - Cigarettes  
  T+ 3:00 2 bowls smoked Cannabis (flowers)
BODY WEIGHT: 190 lb
I had just received my package on Tuesday. I planned on tripping throughout an entire day, getting up in the early hours (5am) to dose one of the two hits I had purchased. I had trouble sleeping and I ended up only getting about 5 hours of sleep because of how excited I was. When I awoke at 5:30 I was more energetic and positive than ever, despite my lack of sleep. I grabbed some coffee, took some vitamin supplements and awaited for 6:00am because I figured it would be better to dose on the hour.

6:00am - Tasteless. Not even the slightest amount of any sort of bitterness or any other tastes. This was my first time ever trying the drug, but after extensive reading I knew this was a good thing. This put a smile on my face, I proceeded to take pictures of t

### Final Version
Works by showing all trip reports and processes the html with Beautiful Soup.

In [64]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium.common.exceptions import TimeoutException

# Initialize WebDriver
driver = webdriver.Safari()
driver.set_page_load_timeout(15)  # Set timeout limit

BASE_URL = "https://www.erowid.org/experiences/exp.cgi?ShowViews=0&Cellar=0&Start=0&Max=39877"
LINKS_FILE = "erowid_links.txt"
REPORTS_FILE = "Erowid_Trip_Reports.csv"
BATCH_SIZE = 100  # Save data in batches

def get_all_report_links(start_url):
    """Fetch all experience report links and save them to LINKS_FILE."""
    if os.path.exists(LINKS_FILE):
        print("Loading existing report links...")
        with open(LINKS_FILE, "r") as f:
            return list(set(f.read().splitlines()))
    
    driver.get(start_url)
    report_links = []
    
    start_time = time.time()
    print(f"Scraping index page: {driver.current_url}")
    page_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'exp.php?ID=')]")
    report_links.extend([link.get_attribute('href') for link in page_links 
                         if link.get_attribute('href').startswith("https://www.erowid.org")])
    
    seconds = time.time() - start_time
    print(f"<<<<<<<<<< Finished gathering links >>>>>>>>>\nTook {seconds:.2f} seconds.")
    
    # Remove duplicates and save to file
    report_links = list(set(report_links))
    with open(LINKS_FILE, "w") as f:
        f.write("\n".join(report_links))
    
    return report_links

def scrape_erowid_reports(report_links):
    """Scrape trip reports and merge new data into REPORTS_FILE without duplicates."""
    all_data = []
    processed_count = 0

    # Load already scraped links from CSV if available
    if os.path.exists(REPORTS_FILE):
        existing_df = pd.read_csv(REPORTS_FILE)
        scraped_links = set(existing_df["Link"].dropna().tolist())
    else:
        scraped_links = set()

    # Remove duplicates from input list and filter out already scraped links
    report_links = list(set(report_links))
    report_links = [link for link in report_links if link not in scraped_links]
    print(f"Remaining reports to scrape: {len(report_links)}")

    start_time = time.time()

    for i, href in enumerate(tqdm(report_links, desc="Scraping Reports", unit="report")):
        # Safety check to ensure link is not processed twice
        if href in scraped_links:
            continue

        if not href.startswith("https://www.erowid.org"):
            print(f"Skipping invalid link: {href}")
            continue

        try:
            driver.get(href)
        except TimeoutException:
            print(f"Timeout error: Skipping {href}")
            continue
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        
        if "403 Forbidden: Your IP Address Has Been Blocked" in page_source:
            ip_address = soup.find("h2").text.split(": ")[-1].strip()
            print(f"🚨 IP BANNED: {ip_address} 🚨")
            print(f"Process stopped. Successfully scraped {processed_count} reports.")
            driver.quit()
            break
        
        if "reset.me" in driver.current_url or "wordpress.com" in driver.current_url:
            print(f"🚨 Redirected to external site: {driver.current_url}. Skipping... 🚨")
            continue

        try:
            title_element = soup.find("div", class_="title")
            title = title_element.text.strip() if title_element else "Unknown Title"
            
            substance_element = soup.find("div", class_="substance")
            substance = substance_element.text.strip() if substance_element else "Unknown Substance"
            
            author_element = soup.find("div", class_="author")
            author = author_element.text.replace("by", "").strip() if author_element else "Unknown"
            
            bodyweight_element = soup.find("td", class_="bodyweight-amount")
            bodyweight = bodyweight_element.text.strip() if bodyweight_element else "Unknown"
            
            dose_chart_entries = []
            for row in soup.select("table.dosechart tbody tr"):
                cols = row.find_all("td")
                if len(cols) == 5:
                    dose_chart_entries.append(
                        " | ".join([col.text.strip() for col in cols])
                    )
            dose_chart = "\n".join(dose_chart_entries) if dose_chart_entries else "No Dose Chart Available"
            
            report_text_element = soup.find("div", class_="report-text-surround")
            report_text = "".join(report_text_element.stripped_strings) if report_text_element else "No Text Available"
            
            all_data.append({
                "Title": title,
                "Substance": substance,
                "Author": author,
                "Bodyweight": bodyweight,
                "Dose Chart": dose_chart,
                "Report Text": report_text,
                "Link": href
            })
            processed_count += 1
            scraped_links.add(href)  # Mark as scraped
        except Exception as e:
            print(f"Error scraping {href}: {e}")
            continue

        # Save in batches
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(report_links):
            df_batch = pd.DataFrame(all_data)
            if os.path.exists(REPORTS_FILE):
                existing_df = pd.read_csv(REPORTS_FILE)
                combined_df = pd.concat([existing_df, df_batch], ignore_index=True)
                combined_df.drop_duplicates(subset="Link", inplace=True)
                combined_df.to_csv(REPORTS_FILE, index=False)
            else:
                df_batch.drop_duplicates(subset="Link", inplace=True)
                df_batch.to_csv(REPORTS_FILE, index=False)
            all_data.clear()
            print(f"Saved batch {(i + 1)} to {REPORTS_FILE}")

    seconds_total = time.time() - start_time
    print(f"Finished scraping {processed_count} reports in {seconds_total:.2f} seconds.")
    driver.quit()

def remove_duplicates_from_csv():
    """Clean the CSV file by removing any duplicate entries based on the Link column."""
    if os.path.exists(REPORTS_FILE):
        df = pd.read_csv(REPORTS_FILE)
        before = len(df)
        df.drop_duplicates(subset="Link", inplace=True)
        df.to_csv(REPORTS_FILE, index=False)
        after = len(df)
        print(f"Removed {before - after} duplicate entries from {REPORTS_FILE}.")

# Main execution steps
report_links = get_all_report_links(BASE_URL)
scrape_erowid_reports(report_links)
remove_duplicates_from_csv()

Loading existing report links...
Remaining reports to scrape: 35061


Scraping Reports:   0%|          | 1/35061 [00:04<42:37:04,  4.38s/report]

🚨 Redirected to external site: https://shivahaoma.wordpress.com/2012/02/27/mahashiva-ratri-night-of-the-great-shiva-ii/. Skipping... 🚨


Scraping Reports:   0%|          | 2/35061 [00:04<19:59:57,  2.05s/report]

🚨 Redirected to external site: https://reset.me/personal-story/how-i-completely-healed-my-panic-attacks-with-psilocybin-mushrooms/. Skipping... 🚨


...

Saved batch 900 to Erowid_Trip_Reports.csv


Scraping Reports:   3%|▎         | 1001/35061 [05:22<3:33:38,  2.66report/s]

Saved batch 1000 to Erowid_Trip_Reports.csv


Scraping Reports:   3%|▎         | 1020/35061 [05:26<3:01:37,  3.12report/s]


KeyboardInterrupt: 

Version with IP switching

In [69]:
import pandas as pd
import time
import os
import subprocess  # For running NordVPN CLI commands
import requests    # To fetch the current IP
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium.common.exceptions import TimeoutException

# Initialize WebDriver
driver = webdriver.Safari()
driver.set_page_load_timeout(15)  # Set timeout limit

BASE_URL = "https://www.erowid.org/experiences/exp.cgi?ShowViews=0&Cellar=0&Start=0&Max=39877"
LINKS_FILE = "erowid_links.txt"
REPORTS_FILE = "Erowid_Trip_Reports.csv"
BATCH_SIZE = 100  # Save data in batches

def get_current_ip():
    """
    Fetches and prints the current public IP address.
    """
    try:
        ip = requests.get("https://api.ipify.org").text
        print("Current IP:", ip)
        return ip
    except Exception as e:
        print("Error fetching current IP:", e)
        return None

def change_ip_via_vpn(server='us'):
    """
    Switch IP by disconnecting and reconnecting using NordVPN CLI.
    Make sure NordVPN CLI is installed and you're logged in.
    """
    print("Current IP before switching:")
    get_current_ip()
    print("Switching VPN connection...")
    subprocess.call(["nordvpn", "disconnect"])
    time.sleep(2)  # Short pause after disconnect
    subprocess.call(["nordvpn", "connect", server])
    time.sleep(10)  # Wait for the VPN connection to take effect
    print("VPN switched. New IP should be in effect:")
    get_current_ip()

def get_all_report_links(start_url):
    """Fetch all experience report links and save them to LINKS_FILE."""
    if os.path.exists(LINKS_FILE):
        print("Loading existing report links...")
        with open(LINKS_FILE, "r") as f:
            return list(set(f.read().splitlines()))
    
    driver.get(start_url)
    report_links = []
    
    start_time = time.time()
    print(f"Scraping index page: {driver.current_url}")
    page_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'exp.php?ID=')]")
    report_links.extend([link.get_attribute('href') for link in page_links 
                         if link.get_attribute('href').startswith("https://www.erowid.org")])
    
    seconds = time.time() - start_time
    print(f"<<<<<<<<<< Finished gathering links >>>>>>>>>\nTook {seconds:.2f} seconds.")
    
    # Remove duplicates and save to file
    report_links = list(set(report_links))
    with open(LINKS_FILE, "w") as f:
        f.write("\n".join(report_links))
    
    return report_links

def scrape_erowid_reports(report_links):
    """Scrape trip reports and merge new data into REPORTS_FILE without duplicates."""
    global driver  # So we can reinitialize driver after VPN switching
    all_data = []
    processed_count = 0

    # Load already scraped links from CSV if available
    if os.path.exists(REPORTS_FILE):
        existing_df = pd.read_csv(REPORTS_FILE)
        scraped_links = set(existing_df["Link"].dropna().tolist())
    else:
        scraped_links = set()

    # Remove duplicates from input list and filter out already scraped links
    report_links = list(set(report_links))
    report_links = [link for link in report_links if link not in scraped_links]
    print(f"Remaining reports to scrape: {len(report_links)}")

    start_time = time.time()

    for i, href in enumerate(tqdm(report_links, desc="Scraping Reports", unit="report")):
        # Safety check to ensure link is not processed twice
        if href in scraped_links:
            continue

        if not href.startswith("https://www.erowid.org"):
            print(f"Skipping invalid link: {href}")
            continue

        # Attempt to load the page, with VPN-switching if IP is blocked
        max_attempts = 3
        attempts = 0
        page_loaded = False
        while attempts < max_attempts and not page_loaded:
            try:
                driver.get(href)
                page_source = driver.page_source
                if "403 Forbidden: Your IP Address Has Been Blocked" in page_source:
                    print(f"🚨 IP BLOCKED on {href}. Attempt {attempts+1} of {max_attempts}.")
                    change_ip_via_vpn('us')
                    driver.quit()
                    driver = webdriver.Safari()
                    driver.set_page_load_timeout(15)
                    attempts += 1
                else:
                    page_loaded = True  # Page loaded successfully
            except TimeoutException:
                print(f"Timeout error: Skipping {href}")
                break

        if not page_loaded:
            print(f"Skipping {href} after {max_attempts} attempts.")
            continue

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        
        if "reset.me" in driver.current_url or "wordpress.com" in driver.current_url:
            print(f"🚨 Redirected to external site: {driver.current_url}. Skipping... 🚨")
            continue

        try:
            title_element = soup.find("div", class_="title")
            title = title_element.text.strip() if title_element else "Unknown Title"
            
            substance_element = soup.find("div", class_="substance")
            substance = substance_element.text.strip() if substance_element else "Unknown Substance"
            
            author_element = soup.find("div", class_="author")
            author = author_element.text.replace("by", "").strip() if author_element else "Unknown"
            
            bodyweight_element = soup.find("td", class_="bodyweight-amount")
            bodyweight = bodyweight_element.text.strip() if bodyweight_element else "Unknown"
            
            dose_chart_entries = []
            for row in soup.select("table.dosechart tbody tr"):
                cols = row.find_all("td")
                if len(cols) == 5:
                    dose_chart_entries.append(
                        " | ".join([col.text.strip() for col in cols])
                    )
            dose_chart = "\n".join(dose_chart_entries) if dose_chart_entries else "No Dose Chart Available"
            
            report_text_element = soup.find("div", class_="report-text-surround")
            report_text = "".join(report_text_element.stripped_strings) if report_text_element else "No Text Available"
            
            all_data.append({
                "Title": title,
                "Substance": substance,
                "Author": author,
                "Bodyweight": bodyweight,
                "Dose Chart": dose_chart,
                "Report Text": report_text,
                "Link": href
            })
            processed_count += 1
            scraped_links.add(href)  # Mark as scraped
        except Exception as e:
            print(f"Error scraping {href}: {e}")
            continue

        # Save in batches
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(report_links):
            df_batch = pd.DataFrame(all_data)
            if os.path.exists(REPORTS_FILE):
                existing_df = pd.read_csv(REPORTS_FILE)
                combined_df = pd.concat([existing_df, df_batch], ignore_index=True)
                combined_df.drop_duplicates(subset="Link", inplace=True)
                combined_df.to_csv(REPORTS_FILE, index=False)
            else:
                df_batch.drop_duplicates(subset="Link", inplace=True)
                df_batch.to_csv(REPORTS_FILE, index=False)
            all_data.clear()
            print(f"Saved batch {(i + 1)} to {REPORTS_FILE}")

    seconds_total = time.time() - start_time
    print(f"Finished scraping {processed_count} reports in {seconds_total:.2f} seconds.")
    driver.quit()

def remove_duplicates_from_csv():
    """Clean the CSV file by removing any duplicate entries based on the Link column."""
    if os.path.exists(REPORTS_FILE):
        df = pd.read_csv(REPORTS_FILE)
        before = len(df)
        df.drop_duplicates(subset="Link", inplace=True)
        df.to_csv(REPORTS_FILE, index=False)
        after = len(df)
        print(f"Removed {before - after} duplicate entries from {REPORTS_FILE}.")

# Main execution steps
print("Starting Erowid scraper. Initial IP:")
get_current_ip()
report_links = get_all_report_links(BASE_URL)
scrape_erowid_reports(report_links)
remove_duplicates_from_csv()

Starting Erowid scraper. Initial IP:
Current IP: 185.207.249.100
Loading existing report links...
Remaining reports to scrape: 33776


Scraping Reports:   0%|          | 1/33776 [00:12<119:01:28, 12.69s/report]

🚨 Redirected to external site: https://shivahaoma.wordpress.com/2012/02/27/mahashiva-ratri-night-of-the-great-shiva-ii/. Skipping... 🚨


Scraping Reports:   0%|          | 2/33776 [00:13<52:49:29,  5.63s/report] 

🚨 Redirected to external site: https://reset.me/personal-story/how-i-completely-healed-my-panic-attacks-with-psilocybin-mushrooms/. Skipping... 🚨


...

Scraping Reports:  13%|█▎        | 4401/33776 [19:12<4:27:25,  1.83report/s]

Saved batch 4400 to Erowid_Trip_Reports.csv


Scraping Reports:  13%|█▎        | 4448/33776 [19:29<2:08:32,  3.80report/s] 

🚨 IP BLOCKED on https://www.erowid.org/experiences/exp.php?ID=87669. Attempt 1 of 3.
Current IP before switching:
Current IP: 185.207.249.100
Switching VPN connection...


FileNotFoundError: [Errno 2] No such file or directory: 'nordvpn'

### Clean up any failed Reports

In [ ]:
# remove no text avaliable
trip_reports = pd.read_csv("Erowid_Trip_Reports.csv")
# Filter out Failed Extractions
trip_reports = trip_reports[trip_reports["Report Text"] != "No Text Available"]
trip_reports = trip_reports.drop_duplicates()
# trip_reports.to_csv("Erowid_Trip_Reports.csv")


10494